## Notebook for generating multicomponent dataset (for Chemprop)

### Splitting: Frequency-II

In [2]:
import warnings
warnings.filterwarnings('ignore')
import requests
#from selenium import webdriver
import os
import json
import csv
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from rdkit import Chem
import rdkit.Chem.rdMolDescriptors as MolDescriptors
import rdkit.Chem.Descriptors as Descriptors

In [3]:
%%bash
pwd
ls -ltr

/Users/riteshk/Library/CloudStorage/Box-Box/Research-postdoc/Collaboration-projects/RCC


total 68760
-rw-------@ 1 riteshk  staff      9318 Sep  7  2022 ood_multi_comp_add.csv
-rw-------@ 1 riteshk  staff     41389 Oct  9  2022 freqI_test_multi_comp_comb.csv
-rw-------@ 1 riteshk  staff     35198 Oct  9  2022 freqI_test_multi_comp_add.csv
-rw-------@ 1 riteshk  staff     43087 Oct  9  2022 freqI_test_multi_comp.csv
-rw-------@ 1 riteshk  staff    546675 Oct  9  2022 freqI_train_multi_comp_comb.csv
-rw-------@ 1 riteshk  staff    591956 Oct  9  2022 freqI_train_multi_comp_add.csv
-rw-------@ 1 riteshk  staff     75499 Oct  9  2022 freqI_val_multi_comp_comb.csv
-rw-------@ 1 riteshk  staff     77069 Oct  9  2022 freqI_val_multi_comp_add.csv
-rw-------@ 1 riteshk  staff     78805 Oct  9  2022 freqI_val_multi_comp.csv
-rw-------@ 1 riteshk  staff    573600 Jan 30  2023 freqI_train_multi_comp.csv
-rw-------@ 1 riteshk  staff   4606069 Jun 21 05:46 Masters_Thesis_2023_Victor_Haugaard_Gustav_Silverstam.pdf
-rw-------@ 1 riteshk  staff     16239 Oct  9 16:05 ood_multi_comp_comb.cs

In [4]:
df = pd.read_csv('all_multi_comp.csv')
df

,solv_1_sm,solv_2_sm,solv_3_sm,solv_4_sm,salt_sm,conductivity_log
0,O=C1OCCO1,COC(=O)OC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,2.459589
1,O=C1OCCO1,CCCCOC(=O)OC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,1.774952
2,O=C1OCCO1,CC(C)COC(=O)OC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,1.686399
3,O=C1OCCO1,CCC(C)OC(=O)OC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,1.871802
4,CCOC(=O)OCC,NaN,NaN,NaN,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.741937
...,...,...,...,...,...,...
10191,CS(=O)C,COCCOC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,1.547563
10192,CS(=O)C,COCCOC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,2.624669
10193,CS(=O)C,COCCOC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,2.772589
10194,CS(=O)C,COCCOC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,2.687847


In [6]:
df_comb = pd.read_csv('all_multi_comp_comb.csv')
df_comb

,solv_comb_sm,salt_sm,conductivity_log
0,O=C1OCCO1.COC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,2.459589
1,O=C1OCCO1.CCCCOC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,1.774952
2,O=C1OCCO1.CC(C)COC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,1.686399
3,O=C1OCCO1.CCC(C)OC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,1.871802
4,CCOC(=O)OCC,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.741937
...,...,...,...
10191,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,1.547563
10192,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,2.624669
10193,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,2.772589
10194,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,2.687847


In [21]:
df_add = pd.read_csv('all_multi_comp_add.csv')
df_add

,solv_ratio_1,solv_ratio_2,solv_ratio_3,solv_ratio_4,mol_wt_solv_1,mol_wt_solv_2,mol_wt_solv_3,mol_wt_solv_4,mol_wt_salt,conc_salt,temperature
0,0.447059,0.552941,0.0,0.0,88.016044,90.031694,0.0,0.0,151.980186,1.0,25.0
1,0.422222,0.577778,0.0,0.0,88.016044,132.078644,0.0,0.0,151.980186,1.0,25.0
2,0.422222,0.577778,0.0,0.0,88.016044,132.078644,0.0,0.0,151.980186,1.0,25.0
3,0.422222,0.577778,0.0,0.0,88.016044,132.078644,0.0,0.0,151.980186,1.0,25.0
4,1.000000,0.000000,0.0,0.0,118.062994,0.000000,0.0,0.0,286.933298,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...
10191,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0,151.980186,0.1,30.0
10192,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0,151.980186,0.5,30.0
10193,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0,151.980186,1.0,30.0
10194,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0,151.980186,1.5,30.0


In [5]:
cols_main = ['solvent_1', 'solvent_2', 'solvent_3', 'solvent_4', 'salt_1', 'conductivity_log']
cols_add = ['conc_salt', 'temperature', 'solv_ratio_1', 'solv_ratio_2', 'solv_ratio_3', 'solv_ratio_4']

In [64]:
## My version of calculating frequency
cols = df.columns
def frequency_count():
    freq_dct = {}
    for i in range(len(df)):
        for j in range(4):
            if df[cols[j]][i] != np.nan:
                freq_dct[df[cols[j]][i]] = freq_dct.get(df[cols[j]][i],0)+1
            else:
                pass
    del freq_dct[np.nan]
    return freq_dct

frequency_count()

{'O=C1OCCO1': 2877,
 'COC(=O)OC': 581,
 'CCCCOC(=O)OC': 1,
 'CC(C)COC(=O)OC': 1,
 'CCC(C)OC(=O)OC': 1,
 'CCOC(=O)OCC': 2487,
 'CCOC(=O)OC(C)F': 14,
 'CC(OC(=O)OCC(F)(F)F)F': 11,
 'FC1COC(=O)O1': 66,
 'C1(C(OC(=O)O1)F)F': 23,
 'CCOC(=O)OC': 704,
 'FC(F)C(F)(F)COC(F)(F)C(F)F': 67,
 'C1C(OC(=O)O1)CC(C(F)(F)F)(C(F)(F)F)F': 15,
 'COC(=O)OCC(F)(F)F': 10,
 'COC(F)(C(F)(F)C(F)(F)F)C(F)(C(F)(F)F)C(F)(F)F': 5,
 'CC(OC(F)(F)C(F)C(F)(F)F)C(F)(F)C(F)C(F)(F)F': 15,
 'CC1COC(=O)O1': 5885,
 'CCOCCOCC': 55,
 'ClCCl': 109,
 'C1CCOC1': 292,
 'Cc1ccccc1': 83,
 'CC1CCCO1': 270,
 'COCCOC': 729,
 'C1COCO1': 153,
 'O=c1occo1': 10,
 'C=CCCCOCC1COC(=O)O1': 5,
 'CO[Si](CCCOCC1COC(=O)O1)(OC)OC': 5,
 'CCO[Si](CCCOCC1COC(=O)O1)(OCC)OCC': 5,
 'C[Si](C)(C)O[Si](C)(C)CCCOCC1COC(=O)O1': 5,
 'CCCC(=O)OC': 6,
 'CN1CCCC1=O': 1,
 'CCCCOCCOCCCC': 19,
 'COP(=O)(OC)OC': 5,
 'O=C1CCCO1': 474,
 'COC(=O)OCCCF': 5,
 'CCCOC(=O)OC': 21,
 'COC(=O)OCCC(F)(F)F': 5,
 'COC(=O)OCC(F)(F)C(F)F': 5,
 'COC(=O)OCC(F)(F)C(F)(F)F': 5,
 'O=C1OCC

In [65]:
dct = frequency_count()
print(len(dct))
x = []
y = []

low_freq_set = set()
mid_freq_set = set()
high_freq_set = set()

#dct[key] >= 5000
for key in dct:
    if dct[key] >= 5000:
        high_freq_set.add(key)

#2100 < dct[key] < 5000
count = 0
for key in dct:
    if 2500 < dct[key] < 5000:
        mid_freq_set.add(key)
        count += 1

#dct[key] <= 2100
for key in dct:
    if dct[key] <= 2500:
        low_freq_set.add(key)

print(len(high_freq_set))
print(len(low_freq_set))
print(len(mid_freq_set))
print(count)

202
1
200
1
1


In [66]:
freq_val = list(dct.values())
freq_val.sort(reverse=True)
freq_val

[5885,
 2877,
 2487,
 729,
 704,
 581,
 474,
 415,
 374,
 292,
 270,
 209,
 193,
 168,
 153,
 128,
 119,
 115,
 109,
 108,
 83,
 80,
 73,
 67,
 66,
 66,
 55,
 55,
 54,
 40,
 40,
 40,
 33,
 31,
 31,
 29,
 28,
 25,
 23,
 21,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 19,
 19,
 18,
 18,
 18,
 18,
 16,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 14,
 14,
 14,
 12,
 11,
 11,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [67]:
def save_val_set(df_comb):
    val_df = pd.DataFrame()
    row_count = 0
    count = 0
    for i in range(len(df)):
        if row_count == 0:
            row_count += 1
            pass
        # elif i in ind_cond_zero:
            # pass
        else:
            test_flag = 0
            for j in range(4):
                if df[cols[j]][i] in low_freq_set:
                    test_flag = 1
                    break
                elif df[cols[j]][i] in mid_freq_set:
                    count += 1
                    test_flag = 2
                            
            if test_flag == 2:
                val_series = pd.DataFrame(df_comb.iloc[i]).transpose()
                val_df = val_df.append(pd.DataFrame(val_series))
    return val_df

In [68]:
val_df_comb = save_val_set(df_comb)
val_df_comb.to_csv('val_multi_comp_comb.csv', index=False)
val_df_comb

,solv_comb_sm,salt_sm,conductivity_log
34,O=C1OCCO1,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,1.931521
35,O=C1OCCO1,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,2.163323
36,O=C1OCCO1,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.336472
37,O=C1OCCO1,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,2.501436
38,O=C1OCCO1,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.470004
...,...,...,...
7576,O=C1OCCO1,[Li+].[O-]Cl(=O)(=O)=O,3.365494
7577,O=C1OCCO1,[Li+].[O-]Cl(=O)(=O)=O,3.365906
7578,O=C1OCCO1,[Li+].[O-]Cl(=O)(=O)=O,3.366135
7579,O=C1OCCO1,[Li+].[O-]Cl(=O)(=O)=O,3.366501


In [69]:
val_df_add = save_val_set(df_add)
val_df_add.to_csv('val_multi_comp_add.csv', index=False)
val_df_add

,solv_ratio_1,solv_ratio_2,solv_ratio_3,solv_ratio_4,mol_wt_solv_1,mol_wt_solv_2,mol_wt_solv_3,mol_wt_solv_4,mol_wt_salt,conc_salt,temperature
34,1.0,0.0,0.0,0.0,88.016044,0.0,0.0,0.0,286.933298,1.0,30.00000
35,1.0,0.0,0.0,0.0,88.016044,0.0,0.0,0.0,286.933298,1.0,40.00000
36,1.0,0.0,0.0,0.0,88.016044,0.0,0.0,0.0,286.933298,1.0,50.00000
37,1.0,0.0,0.0,0.0,88.016044,0.0,0.0,0.0,286.933298,1.0,60.00000
38,1.0,0.0,0.0,0.0,88.016044,0.0,0.0,0.0,286.933298,0.1,25.00000
...,...,...,...,...,...,...,...,...,...,...,...
7576,1.0,0.0,0.0,0.0,88.016044,0.0,0.0,0.0,105.964516,1.0,101.07770
7577,1.0,0.0,0.0,0.0,88.016044,0.0,0.0,0.0,105.964516,1.0,95.22533
7578,1.0,0.0,0.0,0.0,88.016044,0.0,0.0,0.0,105.964516,1.0,92.05241
7579,1.0,0.0,0.0,0.0,88.016044,0.0,0.0,0.0,105.964516,1.0,87.08789


In [70]:
def save_test_set(df_comb):
    test_df = pd.DataFrame()
    row_count = 0
    count = 0
    for i in range(len(df)):
        if row_count == 0:
            row_count += 1
            pass
        # elif i in ind_cond_zero:
            # pass
        else:
            test_flag = 0
            for j in range(4):
                if df[cols[j]][i] in low_freq_set:
                    test_flag = 1
                    break
                elif df[cols[j]][i] in mid_freq_set:
                    count += 1
                    test_flag = 2
                            
            if test_flag == 0:
                test_series = pd.DataFrame(df_comb.iloc[i]).transpose()
                test_df = test_df.append(pd.DataFrame(test_series))
            # csv_writer.writerow(row)
    return test_df

In [76]:
test_df_comb = save_test_set(df_comb)
test_df_comb.to_csv('test_multi_comp_comb.csv', index=False)
test_df_comb

,solv_comb_sm,salt_sm,conductivity_log
127,CC1COC(=O)O1,[Li+].[O-]Cl(=O)(=O)=O,-1.444726
128,CC1COC(=O)O1,[Li+].[O-]Cl(=O)(=O)=O,-1.717515
129,CC1COC(=O)O1,[Li+].[O-]Cl(=O)(=O)=O,-2.104468
130,CC1COC(=O)O1,[Li+].[O-]Cl(=O)(=O)=O,-2.769246
131,CC1COC(=O)O1,[Li+].[O-]Cl(=O)(=O)=O,-3.115396
...,...,...,...
6818,CC1COC(=O)O1,[Li+].[B-](F)(F)(F)F,-2.599317
6819,CC1COC(=O)O1,[Li+].[B-](F)(F)(F)F,-3.205453
6820,CC1COC(=O)O1,[Li+].[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2,-1.236012
6821,CC1COC(=O)O1,[Li+].[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2,-1.147065


In [72]:
test_df_add = save_test_set(df_add)
test_df_add.to_csv('test_multi_comp_add.csv', index=False)
test_df_add

,solv_ratio_1,solv_ratio_2,solv_ratio_3,solv_ratio_4,mol_wt_solv_1,mol_wt_solv_2,mol_wt_solv_3,mol_wt_solv_4,mol_wt_salt,conc_salt,temperature
127,1.0,0.0,0.0,0.0,102.031694,0.0,0.0,0.0,105.964516,0.009736,25.0
128,1.0,0.0,0.0,0.0,102.031694,0.0,0.0,0.0,105.964516,0.007299,25.0
129,1.0,0.0,0.0,0.0,102.031694,0.0,0.0,0.0,105.964516,0.004866,25.0
130,1.0,0.0,0.0,0.0,102.031694,0.0,0.0,0.0,105.964516,0.002434,25.0
131,1.0,0.0,0.0,0.0,102.031694,0.0,0.0,0.0,105.964516,0.001706,25.0
...,...,...,...,...,...,...,...,...,...,...,...
6818,1.0,0.0,0.0,0.0,102.031694,0.0,0.0,0.0,94.018923,2.006981,-30.0
6819,1.0,0.0,0.0,0.0,102.031694,0.0,0.0,0.0,94.018923,2.348000,-30.0
6820,1.0,0.0,0.0,0.0,102.031694,0.0,0.0,0.0,193.984627,0.169485,-30.0
6821,1.0,0.0,0.0,0.0,102.031694,0.0,0.0,0.0,193.984627,0.501409,-30.0


In [73]:
def save_train_set(df_comb):
    train_df = pd.DataFrame()
    row_count = 0
    count = 0
    for i in range(len(df)):
        if row_count == 0:
            row_count += 1
            pass
        # elif i in ind_cond_zero:
            # pass
        else:
            test_flag = 0
            for j in range(4):
                if df[cols[j]][i] in low_freq_set:
                    test_flag = 1
                    break
                elif df[cols[j]][i] in mid_freq_set:
                    count += 1
                    test_flag = 2
                            
            if test_flag == 1:
                train_series = pd.DataFrame(df_comb.iloc[i]).transpose()
                train_df = train_df.append(pd.DataFrame(train_series))
            # csv_writer.writerow(row)
    return train_df

In [74]:
train_df_comb = save_train_set(df_comb)
train_df_comb.to_csv('train_multi_comp_comb.csv', index=False)
train_df_comb

,solv_comb_sm,salt_sm,conductivity_log
1,O=C1OCCO1.CCCCOC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,1.774952
2,O=C1OCCO1.CC(C)COC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,1.686399
3,O=C1OCCO1.CCC(C)OC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,1.871802
4,CCOC(=O)OCC,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.741937
5,CCOC(=O)OCC,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.955511
...,...,...,...
10191,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,1.547563
10192,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,2.624669
10193,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,2.772589
10194,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,2.687847


In [75]:
train_df_add = save_train_set(df_add)
train_df_add.to_csv('train_multi_comp_add.csv', index=False)
train_df_add

,solv_ratio_1,solv_ratio_2,solv_ratio_3,solv_ratio_4,mol_wt_solv_1,mol_wt_solv_2,mol_wt_solv_3,mol_wt_solv_4,mol_wt_salt,conc_salt,temperature
1,0.422222,0.577778,0.0,0.0,88.016044,132.078644,0.0,0.0,151.980186,1.0,25.0
2,0.422222,0.577778,0.0,0.0,88.016044,132.078644,0.0,0.0,151.980186,1.0,25.0
3,0.422222,0.577778,0.0,0.0,88.016044,132.078644,0.0,0.0,151.980186,1.0,25.0
4,1.000000,0.000000,0.0,0.0,118.062994,0.000000,0.0,0.0,286.933298,1.0,10.0
5,1.000000,0.000000,0.0,0.0,118.062994,0.000000,0.0,0.0,286.933298,1.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...
10191,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0,151.980186,0.1,30.0
10192,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0,151.980186,0.5,30.0
10193,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0,151.980186,1.0,30.0
10194,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0,151.980186,1.5,30.0
